In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
#load up all the data
inputfolder = 'data'
filelist = os.listdir(inputfolder)
framedict = dict()
for file in filelist:
    filename,ext = os.path.splitext(file)
    pathway = os.path.join(inputfolder,file)
#     print(pathway)
    tempdf = pd.read_csv(pathway)
    framedict[filename] = tempdf
    del tempdf
dictnames = [x for x in framedict.keys()]
print(dictnames)

['ATP_matches', 'aus_open_men_match_odds', 'aus_open_men_tournament_odds', 'aus_open_women_match_odds', 'aus_open_women_tournament_odds', 'WTA_matches']


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (11,12,13,14,17,18,24,25,26,27,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Looking at the mens data

In [3]:
#load mens dataframes into variables and inspect data
ATPframe = framedict['ATP_matches'].copy() #we copy so we can run the code multiple times without errors, not really necessary
AOMensMatchOdds = framedict[ 'aus_open_men_match_odds'].copy()
AOMensTournOdds = framedict['aus_open_men_tournament_odds'].copy()
display(ATPframe.head())
display(AOMensMatchOdds.head())
display(AOMensTournOdds.head())

#initial cleaning
#checking that we're only looking at mens data
print(AOMensMatchOdds['men_women'].unique())
print(AOMensTournOdds['men_women'].unique())

#only mens, so we can drop this column
AOMensMatchOdds.drop('men_women',axis=1,inplace=True)
AOMensTournOdds.drop('men_women',axis=1,inplace=True)

Winner           Loser Tournament Tournament_Date  \
0  Edouard Roger-Vasselin     Eric Prodon    Chennai        2-Jan-12   
1               Dudi Sela   Fabio Fognini    Chennai        2-Jan-12   
2                Go Soeda   Frederico Gil    Chennai        2-Jan-12   
3            Yuki Bhambri      Karol Beck    Chennai        2-Jan-12   
4           Yuichi Sugita  Olivier Rochus    Chennai        2-Jan-12   

  Court_Surface Round_Description Winner_Rank Loser_Rank  Retirement_Ind  \
0          Hard       First Round         106         97               0   
1          Hard       First Round          83         48               0   
2          Hard       First Round         120        102               0   
3          Hard       First Round         345        101               0   
4          Hard       First Round         235         67               0   

  Winner_Sets_Won          ...           Loser_DoubleFaults  \
0               2          ...                            3   
1               2          ...                            4   
2               2          ...                            2   
3               2          ...                            1   
4               2          ...                            0   

  Loser_FirstServes_Won Loser_FirstServes_In Loser_SecondServes_Won  \
0                    21                   33                     13   
1                    17                   32                      5   
2                    45                   70                     18   
3                    15                   33                     13   
4                    19                   32                     13   

  Loser_SecondServes_In Loser_BreakPoints_Won Loser_BreakPoints  \
0                    26                     1                 3   
1                    26                     0                 1   
2                    35                     2                 4   
3                    29                     2                 3   
4                    22                     1                 7   

  Loser_ReturnPoints_Won Loser_ReturnPoints_Faced Loser_TotalPoints_Won  
0                     15                       49                    49  
1                      8                       33                    30  
2                     36                      103                    99  
3                     15                       46                    43  
4                     30                       78                    62  

[5 rows x 35 columns]

tournament men_women  match_date             player_1  \
0  Australian Open 2012       Men  2012-01-17       Potito Starace   
1  Australian Open 2012       Men  2012-01-17        Nicolas Mahut   
2  Australian Open 2012       Men  2012-01-17     Filippo Volandri   
3  Australian Open 2012       Men  2012-01-17          Lukas Rosol   
4  Australian Open 2012       Men  2012-01-17  Cedrik Marcel Stebe   

   player_1_odds            player_2  player_2_odds  
0           1.77         Tatsuma Ito           2.24  
1           5.30      Radek Stepanek           1.20  
2          40.00        Milos Raonic           1.02  
3           3.25  Philipp Petzschner           1.42  
4           3.60      Lleyton Hewitt           1.36

tournament men_women              player   odds
0  Australian Open 2012       Men      Novak Djokovic   2.46
1  Australian Open 2012       Men       Roger Federer   5.60
2  Australian Open 2012       Men         Andy Murray   7.60
3  Australian Open 2012       Men        Rafael Nadal   9.00
4  Australian Open 2012       Men  Jo Wilfried Tsonga  22.00

['Men']
['Men']


In [15]:
#convert dates to datetime
ATPframe['datetime'] = pd.to_datetime(ATPframe['Tournament_Date'])
AOMensMatchOdds['datetime'] = pd.to_datetime(AOMensMatchOdds['match_date'])
display(AOMensMatchOdds.loc[:,['match_date','datetime']].head())

#testing splitting up dates into day/month/year
AOMensMatchOdds['year'] = pd.DatetimeIndex(AOMensMatchOdds['datetime']).year
AOMensMatchOdds['month'] = pd.DatetimeIndex(AOMensMatchOdds['datetime']).month
AOMensMatchOdds['day'] = pd.DatetimeIndex(AOMensMatchOdds['datetime']).day
ATPframe['year'] = pd.DatetimeIndex(ATPframe['datetime']).year
ATPframe['month'] = pd.DatetimeIndex(ATPframe['datetime']).month
ATPframe['day'] = pd.DatetimeIndex(ATPframe['datetime']).day
display(ATPframe.loc[:,['year','month','day','datetime']].head())

#we only have odds data for Australian Open, so we're gonna just grab ATP data for Australian Opens
print(AOMensTournOdds['tournament'].unique())
print(AOMensTournOdds['tournament'].unique() == AOMensMatchOdds['tournament'].unique())#checking to see if odds data are from the same tournaments

#new column for Aus open names so we can match it
def AusOpen(row):
    tourney = row['Tournament']
    if 'Australian Open' in tourney:
        return 'Ausralian Open '+str(row['year'])
    else:
        return np.nan
ATPframe['AusOpen'] = ATPframe.apply(AusOpen,axis=1)
print(ATPframe['AusOpen'].unique())
AusOpenframe = ATPframe.dropna(subset =['AusOpen'],axis=0)
display(AusOpenframe.head())

match_date   datetime
0  2012-01-17 2012-01-17
1  2012-01-17 2012-01-17
2  2012-01-17 2012-01-17
3  2012-01-17 2012-01-17
4  2012-01-17 2012-01-17

year  month  day   datetime
0  2012      1    2 2012-01-02
1  2012      1    2 2012-01-02
2  2012      1    2 2012-01-02
3  2012      1    2 2012-01-02
4  2012      1    2 2012-01-02

['Australian Open 2012' 'Australian Open 2013' 'Australian Open 2014'
 'Australian Open 2015' 'Australian Open 2016' 'Australian Open 2017'
 'Australian Open 2018']
[ True  True  True  True  True  True  True]
[nan 'Ausralian Open 2012' 'Ausralian Open 2013' 'Ausralian Open 2014'
 'Ausralian Open 2015' 'Ausralian Open 2016' 'Ausralian Open 2017'
 'Ausralian Open 2018']


Winner               Loser                  Tournament  \
137  Adrian Menendez-Maceiras  Alejandro Gonzalez  Australian Open, Melbourne   
138            Marco Crugnola      Amir Weintraub  Australian Open, Melbourne   
139              Sergei Bubka      Andre Begemann  Australian Open, Melbourne   
140           Denys Molchanov       Andrew Harris  Australian Open, Melbourne   
141                Bjorn Phau      Arnaud Clement  Australian Open, Melbourne   

    Tournament_Date Court_Surface Round_Description Winner_Rank Loser_Rank  \
137       16-Jan-12          Hard        Qualifying         195        270   
138       16-Jan-12          Hard        Qualifying         274        220   
139       16-Jan-12          Hard        Qualifying         152        227   
140       16-Jan-12          Hard        Qualifying         267          .   
141       16-Jan-12          Hard        Qualifying         174        157   

     Retirement_Ind Winner_Sets_Won         ...           \
137               0               2         ...            
138               0               2         ...            
139               0               2         ...            
140               0               2         ...            
141               0               2         ...            

     Loser_BreakPoints_Won Loser_BreakPoints Loser_ReturnPoints_Won  \
137                      3                10                     24   
138                      1                 1                     14   
139                      3                 3                     30   
140                      1                 2                     21   
141                      3                10                     32   

    Loser_ReturnPoints_Faced Loser_TotalPoints_Won   datetime  year month day  \
137                       53                    44 2012-01-16  2012     1  16   
138                       43                    46 2012-01-16  2012     1  16   
139                       76                    82 2012-01-16  2012     1  16   
140                       64                    57 2012-01-16  2012     1  16   
141                       73                    63 2012-01-16  2012     1  16   

                 AusOpen  
137  Ausralian Open 2012  
138  Ausralian Open 2012  
139  Ausralian Open 2012  
140  Ausralian Open 2012  
141  Ausralian Open 2012  

[5 rows x 40 columns]

Winner                               object
Loser                                object
Tournament                           object
Tournament_Date                      object
Court_Surface                        object
Round_Description                    object
Winner_Rank                          object
Loser_Rank                           object
Retirement_Ind                        int64
Winner_Sets_Won                      object
Winner_Games_Won                      int64
Winner_Aces                          object
Winner_DoubleFaults                  object
Winner_FirstServes_Won               object
Winner_FirstServes_In                object
Winner_SecondServes_Won              object
Winner_SecondServes_In               object
Winner_BreakPoints_Won               object
Winner_BreakPoints                   object
Winner_ReturnPoints_Won              object
Winner_ReturnPoints_Faced            object
Winner_TotalPoints_Won                int64
Loser_Sets_Won                  

Im using a project completed by Michal SIPKO for one of his computing courses, dunno man i found it on google. Anyways he suggets that one of the easiest ways it to compare two players is to look at the difference between the two. the arguement for looking at differences is so that it ensures a symmetrical model, where it doesnt matter which player we put in as player 1. THis seems to make a lot of sense to me as it ensures that a feature A is weighed the same for both players
<p>
so one of the things we notice about the ATP data is that because it is historic data, it is organised as [WINNER,LOSER]. The problem here, is that the submission template is organised as [Player1, Player2, %Player1Victory]. currently the algorithm will just learn that player1 will always win, so we'll have to randomly pick half of the rows to reverse player order, then split out the winner in a separate column as the label

In [32]:
#so instead of splitting and reversing the frames first, we're gonna calculate the difference and then do the reverse player direction

# get both frame, do some cleaning
winframe = AusOpenframe.filter(regex='Winner')
loseframe = AusOpenframe.filter(regex='Loser')
winlist = [x for x in winframe.columns]
loselist = [x for x in loseframe.columns]
for col in winlist[1:]:
    winframe.loc[:,col] = pd.to_numeric(winframe[col],errors="coerce")
for col in loselist[1:]:
    loseframe.loc[:,col] = pd.to_numeric(loseframe[col],errors="coerce")
colnames = ['_'.join(x.split('_')[1:]) for x in winlist[1:]]
display(winframe.head())
display(loseframe.head())
print(winframe.isna().sum())
print(loseframe.isna().sum())
display(winframe[winframe['Winner_Rank'].isnull()])
AusOpenframe[AusOpenframe['Winner']=='Jordan Thompson']

Winner  Winner_Rank  Winner_Sets_Won  Winner_Games_Won  \
137  Adrian Menendez-Maceiras        195.0                2                12   
138            Marco Crugnola        274.0                2                12   
139              Sergei Bubka        152.0                2                14   
140           Denys Molchanov        267.0                2                13   
141                Bjorn Phau        174.0                2                13   

     Winner_Aces  Winner_DoubleFaults  Winner_FirstServes_Won  \
137          3.0                  1.0                      19   
138          1.0                  3.0                      22   
139          5.0                  2.0                      32   
140          3.0                  2.0                      29   
141          3.0                  0.0                      27   

     Winner_FirstServes_In  Winner_SecondServes_Won  Winner_SecondServes_In  \
137                     31                       10                      22   
138                     29                        7                      14   
139                     45                       14                      31   
140                     38                       14                      26   
141                     39                       14                      34   

     Winner_BreakPoints_Won  Winner_BreakPoints  Winner_ReturnPoints_Won  \
137                     7.0                11.0                     32.0   
138                     5.0                15.0                     36.0   
139                     4.0                13.0                     42.0   
140                     4.0                10.0                     32.0   
141                     6.0                11.0                     35.0   

     Winner_ReturnPoints_Faced  Winner_TotalPoints_Won  
137                       52.0                      61  
138                       68.0                      65  
139                       94.0                      88  
140                       68.0                      75  
141                       66.0                      76

Loser  Loser_Rank  Loser_Sets_Won  Loser_Games_Won  \
137  Alejandro Gonzalez       270.0               0                5   
138      Amir Weintraub       220.0               0                4   
139      Andre Begemann       227.0               0               11   
140       Andrew Harris         NaN               0                6   
141      Arnaud Clement       157.0               0                7   

     Loser_Aces  Loser_DoubleFaults  Loser_FirstServes_Won  \
137         0.0                 3.0                     11   
138         0.0                 1.0                     28   
139         4.0                 8.0                     40   
140         0.0                 1.0                     17   
141         4.0                 3.0                     21   

     Loser_FirstServes_In  Loser_SecondServes_Won  Loser_SecondServes_In  \
137                    28                       9                     24   
138                    59                       4                      9   
139                    58                      12                     36   
140                    37                      19                     31   
141                    39                      10                     27   

     Loser_BreakPoints_Won  Loser_BreakPoints  Loser_ReturnPoints_Won  \
137                    3.0               10.0                    24.0   
138                    1.0                1.0                    14.0   
139                    3.0                3.0                    30.0   
140                    1.0                2.0                    21.0   
141                    3.0               10.0                    32.0   

     Loser_ReturnPoints_Faced  Loser_TotalPoints_Won  
137                      53.0                     44  
138                      43.0                     46  
139                      76.0                     82  
140                      64.0                     57  
141                      73.0                     63

Winner                         0
Winner_Rank                    8
Winner_Sets_Won                0
Winner_Games_Won               0
Winner_Aces                    2
Winner_DoubleFaults           10
Winner_FirstServes_Won         0
Winner_FirstServes_In          0
Winner_SecondServes_Won        0
Winner_SecondServes_In         0
Winner_BreakPoints_Won         3
Winner_BreakPoints             3
Winner_ReturnPoints_Won      143
Winner_ReturnPoints_Faced    143
Winner_TotalPoints_Won         0
dtype: int64
Loser                         0
Loser_Rank                   34
Loser_Sets_Won                0
Loser_Games_Won               0
Loser_Aces                    2
Loser_DoubleFaults            9
Loser_FirstServes_Won         0
Loser_FirstServes_In          0
Loser_SecondServes_Won        0
Loser_SecondServes_In         0
Loser_BreakPoints_Won         8
Loser_BreakPoints             4
Loser_ReturnPoints_Won      143
Loser_ReturnPoints_Faced    143
Loser_TotalPoints_Won         0
dtype: int64

Winner  Winner_Rank  Winner_Sets_Won  Winner_Games_Won  \
3153   Jordan Thompson          NaN                2                19   
8946      Marc Polmans          NaN                2                13   
11885  Oliver Anderson          NaN                2                12   
14860    Matthew Ebden          NaN                2                12   
14892        Alex Bolt          NaN                2                18   
14931        Alex Bolt          NaN                2                13   
14960        Alex Bolt          NaN                2                13   
17949  James Duckworth          NaN                2                12   

       Winner_Aces  Winner_DoubleFaults  Winner_FirstServes_Won  \
3153           5.0                  5.0                      54   
8946           1.0                  0.0                      24   
11885          7.0                  1.0                      30   
14860          3.0                  1.0                      29   
14892         10.0                  4.0                      50   
14931         10.0                  0.0                      32   
14960         13.0                  1.0                      35   
17949         12.0                  1.0                      30   

       Winner_FirstServes_In  Winner_SecondServes_Won  Winner_SecondServes_In  \
3153                      71                       15                      40   
8946                      32                       12                      20   
11885                     44                       16                      24   
14860                     30                        4                       8   
14892                     62                       22                      37   
14931                     45                       17                      27   
14960                     43                       14                      28   
17949                     40                       10                      18   

       Winner_BreakPoints_Won  Winner_BreakPoints  Winner_ReturnPoints_Won  \
3153                      5.0                13.0                     43.0   
8946                      5.0                11.0                     37.0   
11885                     3.0                 6.0                     27.0   
14860                     4.0                 6.0                     25.0   
14892                     2.0                11.0                     42.0   
14931                     3.0                10.0                     38.0   
14960                     2.0                 5.0                     16.0   
17949                     3.0                 5.0                     23.0   

       Winner_ReturnPoints_Faced  Winner_TotalPoints_Won  
3153                       110.0                     112  
8946                        80.0                      73  
11885                       62.0                      73  
14860                       48.0                      58  
14892                      117.0                     114  
14931                       91.0                      87  
14960                       53.0                      65  
17949                       58.0                      65

Winner             Loser                  Tournament  \
3153   Jordan Thompson  Nicolas Renavand  Australian Open, Melbourne   
14998  Jordan Thompson        Joao Sousa  Australian Open, Melbourne   

      Tournament_Date Court_Surface Round_Description Winner_Rank Loser_Rank  \
3153        14-Jan-13          Hard        Qualifying           .        273   
14998       16-Jan-17          Hard       First Round          76         37   

       Retirement_Ind Winner_Sets_Won         ...           \
3153                0               2         ...            
14998               0               3         ...            

       Loser_BreakPoints_Won Loser_BreakPoints Loser_ReturnPoints_Won  \
3153                       3                11                     42   
14998                      3                11                     49   

      Loser_ReturnPoints_Faced Loser_TotalPoints_Won   datetime  year month  \
3153                       116                   106 2013-01-14  2013     1   
14998                      146                   129 2017-01-16  2017     1   

      day              AusOpen  
3153   14  Ausralian Open 2013  
14998  16  Ausralian Open 2017  

[2 rows x 40 columns]

In [ ]:
def get_difference(frame1,frame2,colname=None, prefix1='',prefix2='',index1=None,index2=None):
    #check if index between frames are identical, if not, reset both
    idx_check = frame1.index.equals(frame2.index)
    if not(idx_check):
        frame1.reset_index(drop=True, inplace=True)
        frame2.reset_index(drop=True, inplace=True)
    if colname is None:
        colname = frame1.columns
    if frame1.shape[0] != frame2.shape[0]:
        raise ValueError('both frames must contain the same number of rows/n Frame1: %{}/mn Frame2: %{}'.format([frame1.shape[0],frame2.shape[0]]))
    tempdiffcolnames = [x+"_diff" for x in colname]
    diffcolnames = list()
    if index1 is not None:
        diffcolnames.append(index1)
    if index2 is not None:
        diffcolnames.append(index2)
    diffcolnames = diffcolnames + tempdiffcolnames
    if idx_check:
        difframe = pd.DataFrame(index=frame1.index,columns=diffcolnames,data=0)
    else:
        difframe = pd.DataFrame(index=range(0,frame1.shape[0]),columns=diffcolnames,data=0)
    if index1 is not None:
        difframe[index1] = frame1[index1]
    if index2 is not None:
        difframe[index2] = frame2[index2]
    for col in colname:
        frame1name = prefix1+col
        frame2name = prefix2+col
        difframename = col+"_diff"
        difframe[difframename]=frame1[frame1name] - frame2[frame2name]
    return difframe
difdf = get_difference(winframe,loseframe,colname = colnames,prefix1="Winner_",prefix2="Loser_",index1="Winner",index2="Loser")
display(difdf.head())
